###  Задание 1.

Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [164]:
import requests

In [165]:
from bs4 import BeautifulSoup

In [201]:
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/all/'
params = {
    'search_query': 'python'
}

In [202]:
req= requests.get(URL,params)
req

<Response [200]>

In [203]:
req.text

'<!DOCTYPE html>\n<html lang="ru" class="no-js">\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta content=\'width=1024\' name=\'viewport\'>\n<title>Все публикации подряд / Хабр</title>\n\n\n\n  <meta property="fb:app_id" content="444736788986613" />\n<meta property="og:type" content="website"/>\n<meta property="fb:pages" content="472597926099084"/>\n<meta property="og:site_name" content="Хабр" />\n<link rel="image_src" href="https://habrastorage.org/storage/stuff/habr/habr_ru.png" />\n<meta property="og:image" content="https://habrastorage.org/storage/stuff/habr/habr_ru.png" />\n<meta property="og:image:width" content="1200" />\n<meta property="og:image:height" content="628" />\n<meta property="og:title" content="Все публикации подряд / Хабр"/>\n<meta property="og:description" content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом."/>\n\n\n\n<meta name=\'yandex-verification\

In [204]:
# soup = BeautifulSoup(req.text, 'html.parser')
soup = BeautifulSoup(req.text)
soup                                 

<!DOCTYPE html>
<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Все публикации подряд / Хабр</title>
<meta content="444736788986613" property="fb:app_id"/>
<meta content="website" property="og:type"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="Хабр" property="og:site_name"/>
<link href="https://habrastorage.org/storage/stuff/habr/habr_ru.png" rel="image_src"/>
<meta content="https://habrastorage.org/storage/stuff/habr/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="628" property="og:image:height"/>
<meta content="Все публикации подряд / Хабр" property="og:title"/>
<meta content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом." property="og:description"/>
<meta content="71593b225aeafc4e" name="yandex-verification"/>
<meta content="uns

In [205]:
#posts = soup.find_all('article', class_='post')
#posts
news_blocks = soup.find_all('article', class_='post')
#len(news_blocks)
news_blocks

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/arina27/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/33f/250/a7c/33f250a7c457bb26cd8ddb126d95ee8b.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">arina27</span>
 </a>
 <span class="post__time">сегодня в 14:41</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/558836/">Мир статистических гипотез</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/statistics/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Статистика в IT'); }" rel="nofollow" title="Вы не подписаны на этот хаб">Статистика в IT</a>
 </li>
 <

In [206]:
# добираемся до текста со ссылкой
articles_intro = list(map(lambda x: x.find('div', class_='post__body'), news_blocks))
articles_intro

[<div class="post__body post__body_crop">
 <div class="post__body-cover post__body-cover_fit-cover">
 <img src="https://habrastorage.org/getpro/habr/upload_files/749/48d/26c/74948d26c6a4af0af1641b2edeb5f5a3.jpg" style="object-position: 0% 0%"/>
 </div>
 <div class="post__text post__text-html post__text_v2">
 <p>В современном мире мы обладаем все большим и большим объемом данных о событиях, происходящих вокруг. Зачастую у нас появляются вопросы, на которые хотелось бы быстро ответить на основе имеющейся информации, для этого как нельзя лучше подходит процесс, связанный с проверкой статистических гипотез. Однако, многие считают, что это занятие подразумевает под собой большое число вычислений и в принципе довольно сложно для понимания. На самом деле, алгоритм проверки гипотез достаточно прост, а для осуществления расчетов с каждым годом появляется все больше и больше готовых инструментальных средств, не требующих от человека глубоких познаний в области. Далее я попытаюсь показать, что ма

In [207]:
# добираемся до ссылок
a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
a_list

['https://habr.com/ru/post/558836/#habracut',
 'https://habr.com/ru/post/558828/#habracut',
 'https://pbs.twimg.com/media/EmiZDW3XUAANsJ2.jpg',
 'https://habr.com/ru/post/558826/#habracut',
 'https://habr.com/ru/company/macloud/blog/558316/#habracut',
 'https://habr.com/ru/company/ruvds/blog/556744/',
 'https://habr.com/ru/post/460815/',
 'https://www.amazon.com/gp/product/B06XPJML5D/ref=dbs_a_def_rwt_hsch_vapi_tkin_p1_i0',
 'https://habr.com/ru/post/558820/#habracut',
 'https://habr.com/ru/post/558818/#habracut',
 'https://habr.com/ru/post/558816/#habracut',
 'https://habr.com/ru/company/audiomania/blog/557886/',
 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B0%D1%81%D0%BE%D1%87%D0%BD%D1%8B%D0%B9_%D1%80%D0%B5%D0%B6%D0%B8%D0%BC',
 'https://www.audiomania.ru/content/art-7405.html',
 'https://habr.com/ru/post/558802/#habracut',
 'https://habr.com/ru/post/558794/#habracut',
 'https://habr.com/ru/company/parallels/blog/558786/#habracut',
 'https://habr.com/ru/post/558792/#habracut',
 'https://

In [208]:
for post in news_blocks:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    print('post', post_id)
    hubs = post.find_all('a', class_='hub-link')

post 558836
post 558828
post 558812
post 558826
post 558316
post 556744
post 558822
post 558248
post 558820
post 558818
post 558816
post 556220
post 558442
post 555562
post 558802
post 558794
post 558786
post 558792
post 554276
post 558782


In [210]:
# очень часто сайты могут ограничивать частые запросы к себе, 
# поэтому нужно задерживать исполнение
import time
time.sleep(0.2)

In [292]:
# но мы же собрали только одну страницу? Хотим ВСЕ новости
#KEYWORDS = ['python', 'парсинг']
#URL = 'https://habr.com/ru/all/'
#params = {
#    'search_query': 'python'
#}

def get_all_links(URL, KEYWORDS, pages):
    all_refs = []
    params = {
        'search_query': KEYWORDS
    }
    for i in range(1, pages):
        params['page'] = i
        res = requests.get(URL, params)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        news_blocks = soup.find_all('article', class_='post')
        articles_intro = list(map(lambda x: x.find('div', class_='post__body'), news_blocks))
        a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
        all_refs = a_list
    return all_refs

all_links = get_all_links(URL, 'python', 5)
all_links[]

'https://habr.com/ru/post/558852/#habracut'

In [225]:
import pandas as pd

In [350]:
# получаем ошибочку. Значит не у всех получаемых страниц одинаковая разметка
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    if soup.find('span', class_='post__time'):
        date = soup.find('span', class_='post__time').text
    print(date)
    if soup.find('h2', class_='article_name'): 
        title = soup.find('h2', class_='post__title')
    else: 
        title = soup.find('h1', class_='post__title')
    print(title)
    if soup.find('h2', class_='post__title'):
        title_element = soup.find('a',class_='post__title_link').text
    print(title_element)
    

сегодня в 16:56
<h1 class="post__title post__title_full">
<span class="post__title-text">Современный подход к работе с куки</span>
</h1>
https://habr.com/ru/
сегодня в 16:51
<h1 class="post__title post__title_full">
<span class="post__title-text">Троян в CS-Cart. Утечка счетов из 35'000 интернет-магазинов</span>
</h1>
https://habr.com/ru/
3 ноября 2017 в 13:57
<h1 class="post__title post__title_full">
<span class="post__title-text">Вышел перевод книги Нила Стивенсона «Семиевие»</span>
</h1>
https://habr.com/ru/
сегодня в 16:01
<h1 class="post__title post__title_full">
<span class="post__title-text">Как мы пришли к релизам мобильных приложений раз в неделю</span>
</h1>
https://habr.com/ru/
сегодня в 15:50
<h1 class="post__title post__title_full">
<span class="post__title-text">Рождение современного человека</span>
</h1>
https://habr.com/ru/
сегодня в 15:40
<h1 class="post__title post__title_full">
<span class="post__title-text">Эффект Холла: что это, зачем используется и где применяется

In [355]:
# запишем данные в датафрейм
kom_news = pd.DataFrame()
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    if soup.find('span', class_='post__time'):
        date = soup.find('span', class_='post__time').text
    #print(date)
    if soup.find('h2', class_='article_name'): 
        title = soup.find('h2', class_='post__title')
    else: 
        title = soup.find('h1', class_='post__title')
    #print(title)
    if soup.find('h2', class_='post__title'):
        title_element = soup.find('a',class_='post__title_link').text
    #print(title_element)
    row = {'date': date, 'title': title, 'link': link}
    kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news.reset_index()

,index,date,title,link
0,0,сегодня в 16:56,"[\n, [Современный подход к работе с куки], \n]",https://habr.com/ru/company/ruvds/blog/556742/
1,0,сегодня в 16:51,"[\n, [Троян в CS-Cart. Утечка счетов из 35'000...",https://habr.com/ru/post/558852/#habracut
2,0,3 ноября 2017 в 13:57,"[\n, [Вышел перевод книги Нила Стивенсона «Сем...",https://habr.com/ru/post/373961/
3,0,сегодня в 16:01,"[\n, [Как мы пришли к релизам мобильных прилож...",https://habr.com/ru/company/ozontech/blog/5587...
4,0,сегодня в 15:50,"[\n, [Рождение современного человека], \n]",https://habr.com/ru/post/558840/#habracut
5,0,сегодня в 15:40,"[\n, [Эффект Холла: что это, зачем используетс...",https://habr.com/ru/company/selectel/blog/5559...
6,0,сегодня в 14:41,"[\n, [Мир статистических гипотез], \n]",https://habr.com/ru/post/558836/#habracut
7,0,сегодня в 13:19,"[\n, [Следует ли мне бросить видеоигры?], \n]",https://habr.com/ru/post/558828/#habracut
8,0,сегодня в 13:19,None,https://pbs.twimg.com/media/EmiZDW3XUAANsJ2.jpg
9,0,сегодня в 13:04,"[\n, [3D teeth instance segmentation. В темнот...",https://habr.com/ru/post/558826/#habracut


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

### Задание 2

#### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

#### Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [358]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
EMAIL_2 = {'emailAddresses':['xxx@x.ru']}
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

HEADERS = {
    'Vaar-Version': '0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Header-App-Build-Version': '1.0.0',
}
email_check = requests.post(URL, json=EMAIL_2, headers=HEADERS)
email_check.text

#https://www.avast.com/c-identity-theft

'{"breaches":{"16613":{"breachId":16613,"site":"verifications.io","recordsCount":677914246,"description":"Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\\r\\n\\r\\n","publishDate":"2019-03-28T00:00:00Z","statistics":{"usernames":0,"passwords":0,"emails":677914246}},"17609":{"breachId":17609,"site":"vk.com","recordsCount":42536910,"description":"At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and additional personal information. This breach is

In [363]:
soup = BeautifulSoup((email_check).text, 'html.parser')
soup  

#https://www.avast.com/api/v1/tooltip?locale=en-us

{"breaches":{"16613":{"breachId":16613,"site":"verifications.io","recordsCount":677914246,"description":"Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\r\n\r\n","publishDate":"2019-03-28T00:00:00Z","statistics":{"usernames":0,"passwords":0,"emails":677914246}},"17609":{"breachId":17609,"site":"vk.com","recordsCount":42536910,"description":"At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and additional personal information. This breach is bein

In [360]:
news_blocks_1 = soup.find_all('article', class_='post')
#len(news_blocks)
news_blocks_1

[]

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>